<a href="https://colab.research.google.com/github/ChethaniDilhari/Sri-Lankan-Sign-Language-to-text-Converter/blob/main/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [34]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [35]:
import os
root_dir = "/content/drive/My Drive/"
project_folder = "Colab Notebooks/RealTimeObjectDetection/"
def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)
  
create_and_set_working_directory(project_folder)

In [144]:
WORKSPACE_PATH = root_dir + project_folder +'Tensorflow/workspace'    
SCRIPTS_PATH = root_dir + project_folder +'Tensorflow/scripts'
APIMODEL_PATH = root_dir + project_folder +'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [25]:
print(ANNOTATION_PATH)

/content/drive/My Drive/Colab Notebooks/RealTimeObjectDetection/Tensorflow/workspace/annotations


# 1. Create Label Map

In [37]:
labels = [{'name':'hello', 'id':1}, {'name':'yes', 'id':2},{'name':'no', 'id':3}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [38]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '\train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '\test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

python3: can't open file '/content/drive/My': [Errno 2] No such file or directory
python3: can't open file '/content/drive/My': [Errno 2] No such file or directory


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [58]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 60474, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 60474 (delta 124), reused 177 (delta 74), pack-reused 60221
Receiving objects: 100% (60474/60474), 573.92 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (42016/42016), done.
Checking out files: 100% (2613/2613), done.


In [9]:
pip install wget


In [39]:
import wget

In [47]:
wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#moving file to pre trained model path
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#unzipping
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar (1).gz'

# 4. Copy Model Config to Training Folder

In [41]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [42]:
#!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
#copy pipeline.config in ssd_mobilenet folder to pre trained model path
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

cp: target 'Notebooks/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet' is not a directory


In [55]:
#dependecies for object detection api
from google.protobuf import text_format
!apt-get install -y -qq protobuf-compiler python-pil python-lxml




In [116]:
#!sudo apt install -y protobuf-compiler
%cd {APIMODEL_PATH+ '/research/object_detection/packages/'}


/content/drive/My Drive/Colab Notebooks/RealTimeObjectDetection/Tensorflow/models/research/object_detection/packages
object_detection/protos/*.proto: No such file or directory


In [137]:
%cd {APIMODEL_PATH+ '/research/'}
!protoc object_detection/protos/*.proto --python_out=.
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI


/content/drive/My Drive/Colab Notebooks/RealTimeObjectDetection/Tensorflow/models/research
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-req-build-0i7yzg6v
  Running command git clone -q https://github.com/philferriere/cocoapi.git /tmp/pip-req-build-0i7yzg6v
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264162 sha256=ff176e8799e9cde52f7c85f8c93f6ca0ae7cb9233d9a5900b910aa64459c8dd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-_j9ev3hs/wheels/6b/c6/c5/cb6da4cb793a6cb1ab91f6578d76c42686422127eb4dbcea94
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [110]:
!cp tf2/setup.py . 
!python -m pip install .

Processing /content/drive/My Drive/Colab Notebooks/RealTimeObjectDetection/Tensorflow/models/research/object_detection/packages
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1195 sha256=51b9ce87763327f925aa0a5666f26cc2a7b7efa5cb4e65085e90efc874b435e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-8wvz7bf8/wheels/9f/05/c0/086c67e09e6ce54bad287f22061eb96b65dd2805431047e14a
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Success

# set up path

In [142]:
%cd {APIMODEL_PATH+ '/research/'}
!python object_detection/builders/model_builder_tf2_test.py .

/content/drive/My Drive/Colab Notebooks/RealTimeObjectDetection/Tensorflow/models/research
Traceback (most recent call last):
  File "object_detection/builders/model_builder_tf2_test.py", line 25, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


# 5. Update Config For Transfer Learning

In [111]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ModuleNotFoundError: ignored

In [24]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [25]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

NameError: ignored

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [ ]:
import cv2 
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
cap.release()

In [ ]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt